# **Importing**

In [135]:
import numpy as np
import pandas as pd

In [136]:
import copy

# **Data**

In [137]:
data = pd.DataFrame(columns = ['color', 'outline', 'has_dot', 'shape'])

In [138]:
def create_data(location, color, outline, has_dot, shape):
  data.loc[location] = pd.Series({'color': color, 'outline': outline, 'has_dot': has_dot, 'shape': shape})
  return data

In [139]:
data.loc[0] = pd.Series({'color': 'Green', 'outline': 'Dashed', 'has_dot': 'No', 'shape': 'Triangle'})
data.loc[1] = pd.Series({'color': 'Green', 'outline': 'Dashed', 'has_dot': 'Yes', 'shape': 'Triangle'})
data.loc[2] = pd.Series({'color': 'Yellow', 'outline': 'Dashed', 'has_dot': 'No', 'shape': 'Square'})
data.loc[3] = pd.Series({'color': 'Red', 'outline': 'Dashed', 'has_dot': 'No', 'shape': 'Square'})
data.loc[4] = pd.Series({'color': 'Red', 'outline': 'Solid', 'has_dot': 'No', 'shape': 'Square'})
data.loc[5] = pd.Series({'color': 'Red', 'outline': 'Solid', 'has_dot': 'Yes', 'shape': 'Triangle'})
data.loc[6] = pd.Series({'color': 'Green', 'outline': 'Solid', 'has_dot': 'No', 'shape': 'Square'})
data.loc[7] = pd.Series({'color': 'Green', 'outline': 'Dashed', 'has_dot': 'No', 'shape': 'Triangle'})
data.loc[8] = pd.Series({'color': 'Yellow', 'outline': 'Solid', 'has_dot': 'Yes', 'shape': 'Square'})
data.loc[9] = pd.Series({'color': 'Red', 'outline': 'Solid', 'has_dot': 'No', 'shape': 'Square'})
data.loc[10] = pd.Series({'color': 'Green', 'outline': 'Solid', 'has_dot': 'Yes', 'shape': 'Square'})
data.loc[11] = pd.Series({'color': 'Yellow', 'outline': 'Dashed', 'has_dot': 'Yes', 'shape': 'Square'})
data.loc[12] = pd.Series({'color': 'Yellow', 'outline': 'Solid', 'has_dot': 'No', 'shape': 'Square'})
data.loc[13] = pd.Series({'color': 'Red', 'outline': 'Dashed', 'has_dot': 'Yes', 'shape': 'Triangle'})

data

,color,outline,has_dot,shape
0,Green,Dashed,No,Triangle
1,Green,Dashed,Yes,Triangle
2,Yellow,Dashed,No,Square
3,Red,Dashed,No,Square
4,Red,Solid,No,Square
5,Red,Solid,Yes,Triangle
6,Green,Solid,No,Square
7,Green,Dashed,No,Triangle
8,Yellow,Solid,Yes,Square
9,Red,Solid,No,Square


# **Functions**

In [140]:
def calSum(the_list):
  sum = 0

  for i in range(len(the_list)):
    sum += the_list[i]

  return sum

In [141]:
def calProbability(data_df, detail_df, feature):
  pList = []
  value_count = []
  values = pd.unique(data_df[feature])

  for i in range(len(values)):
    value_count.append(data_df[feature].value_counts()[values[i]])

  total = calSum(value_count)

  for i in range(len(value_count)):
    detail_df[feature + ' = ' + values[i]] = [value_count[i] / total]

  return detail_df

In [142]:
def calProbabilityOfColumnsOf(data_df, detail_df):
  for column in data_df.columns:
    calProbability(data_df, detail_df, column)

  return detail_df

In [143]:
def createDetails(data_df):
  details = pd.DataFrame()
  return calProbabilityOfColumnsOf(data_df, details)

In [144]:
def calInfo(probabilityList):
  entr = 0
  for p in probabilityList:
    entr += p * np.log2(p)

  return entr * (-1)

In [145]:
def getValuesOfA(data_df, column, round):
  return [data_df.at[round, col] for col in data_df.columns if column in col]

In [146]:
def calInfoFor(data_df, column, round):
  p = getValuesOfA(data_df, column, round)

  return calInfo(p)

In [147]:
def calI(data_df, column, round = 0):
  info = []
  for col in pd.unique(data_df[column]):
    candidate = data_df[data_df[column] == col].reset_index(drop=True)

    dataDetails = pd.DataFrame()
    calProbabilityOfColumnsOf(candidate[[column, 'shape']], dataDetails)

    info.append(calInfoFor(dataDetails, 'shape', round))

  return info

In [148]:
def calIRes(probability, info):
  sum = 0

  for i in range(len(info)):
    sum += float(probability[i]) * float(info[i])

  return sum

In [149]:
def calIg(info, iRes):
  return float(info - iRes)

In [150]:
def whatToChoose(data_df, details, target, info, round = 0):
  array = []
  for column in data_df.columns:
    if column == target:
      return max(array), data_df.columns[np.array(array).argmax()]
    array.append(calIg(info, calIRes(getValuesOfA(details, column, round), calI(data_df, column, round))))

  return max(array), data_df.columns[np.array(array).argmax()]

In [151]:
def createSubDf(df, feature, value):
  return df[df[feature] == value].reset_index(drop = True)

In [152]:
def checkEqualSign(string):
  if '=' in string:
    return string.split(' = ')[0]
  return string

In [153]:
def removeFromColumns(columns, string = ' '):
  if string in columns:
    columns.remove(string)
  return columns

In [154]:
def checkEndOfFeaturs(detail_df, data_df, round):
  columns = data_df.columns.tolist()

  for tup in detail_df.at[round, 'choice']:
    string_tup = str(tup)
    string_tup = checkEqualSign(string_tup)
    columns = removeFromColumns(columns, string_tup)

  return len(columns) == 0

In [155]:
def checkEndCondOfBranch(data_df):
  if len(pd.unique(data_df['shape'])) == 1:
    return True
  return False

In [156]:
def setResult(data_df, detail_df, round):
  if checkEndCondOfBranch(data_df) or checkEndOfFeaturs(detail_df, data_df, round):
    next = detail_df.at[round, 'choice'][-1].split(' = ')[0]
    detail_df.at[round, 'choice'][-1] = next + ' = ' + pd.unique(data_df[next])[0]
    detail_df.at[round, 'result'] = pd.unique(data_df['shape'])[0]
  else:
    detail_df.at[round, 'result'] = 'no class'

  detail_df.fillna(value = 'no class', inplace = True)

  return detail_df

In [157]:
def setChoice(detail_df, index, choice):
  detail_df.at[index, 'choice'] = choice
  return detail_df

In [185]:
def updateDetails(detail_df, candidate_df, index = 0, choice = 0):
  detail_df = pd.concat([detail_df, createDetails(candidate_df)], ignore_index = True)
  if index == 0:
    detail_df = setChoice(detail_df, len(detail_df) - 1, [data_details.at[index, 'choice'] + ' = ' + pd.unique(candidate_df[choice])[0]])
  else:
    detail_df = setChoice(detail_df, len(detail_df) - 1, copy.deepcopy(detail_df.at[index, 'choice']))
    detail_df.at[len(detail_df) - 1, 'choice'].append(choice + ' = ')
  detail_df = detail_df.loc[: 'choice'].fillna(0)

  return detail_df

In [179]:
def checkResult(detail_df):
  return not detail_df.at[len(detail_df) - 1, 'result'] == 'no class'

In [191]:
def createResult(data_df, detail_df, choice, index, val):
  candidate = createSubDf(data_df, choice, val)
  detail_df = updateDetails(detail_df, candidate, index, choice)
  setResult(candidate, detail_df, len(detail_df) - 1)

  return candidate, detail_df

In [221]:
def calChoice(data_df, detail_df, target):
  (ig_value, choice) = whatToChoose(data_df, detail_df, target, calInfoFor(detail_df, target, len(detail_df) - 1))
  root_index = len(detail_df) - 1
  possible_choice_values = pd.unique(data_df[choice])

  if root_index == 0:
    detail_df = setChoice(detail_df, root_index, [choice])

  return possible_choice_values, choice, root_index

In [227]:
def goToTheLoop(data_df, detail_df, target):
  possible_choice_values, choice, root_index = calChoice(data_df, detail_df, target)

  for possible_value in possible_choice_values:
    candidate, detail_df = createResult(data_df, detail_df, choice, root_index, possible_value)

    if checkResult(detail_df): continue
    detail_df = goToTheLoop(candidate, detail_df, target)

  return detail_df

# **The code**

In [281]:
target = 'shape'

goToTheLoop(data, createDetails(data), target)

,color = Green,color = Yellow,color = Red,outline = Dashed,outline = Solid,has_dot = No,has_dot = Yes,shape = Triangle,shape = Square,choice,result
0,0.357143,0.285714,0.357143,0.500000,0.500000,0.571429,0.428571,0.357143,0.642857,color,no class
1,1.000000,0.000000,0.000000,0.600000,0.400000,0.600000,0.400000,0.600000,0.400000,[color = Green],no class
2,1.000000,0.000000,0.000000,1.000000,0.000000,0.666667,0.333333,1.000000,0.000000,"[color = Green, outline = Dashed]",Triangle
3,1.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.500000,0.000000,1.000000,"[color = Green, outline = Solid]",Square
4,0.000000,1.000000,0.000000,0.500000,0.500000,0.500000,0.500000,0.000000,1.000000,[color = Yellow],Square
5,0.000000,0.000000,1.000000,0.400000,0.600000,0.600000,0.400000,0.400000,0.600000,[color = Red],no class
6,0.000000,0.000000,1.000000,0.333333,0.666667,1.000000,0.000000,0.000000,1.000000,"[color = Red, has_dot = No]",Square
7,0.000000,0.000000,1.000000,0.500000,0.500000,0.000000,1.000000,1.000000,0.000000,"[color = Red, has_dot = Yes]",Triangle


# **The tree**

In [244]:
!pip install treelib

In [245]:
from treelib import Node, Tree

In [246]:
def cNode(tree, nodeName, node, parent):
  if(parent != 'NoParent'):
    tree.create_node(nodeName, node, parent)
  else:
    tree.create_node(nodeName, node)

  return tree

In [247]:
data_details = goToTheLoop(data, createDetails(data), target)

In [280]:
decisionTree = Tree()

for choice in data_details.choice:
  if type(choice) == str:
    cNode(decisionTree, choice, 'root', 'NoParent')
  else:
    if len(choice) == 1:
      cNode(decisionTree, choice[-1].split(' = ')[1], str(choice), 'root')
    else:
      cNode(decisionTree, choice[-1].split(' = ')[0] + ' ___ ' + choice[-1].split(' = ')[1], str(choice), str(choice[:-1]))

decisionTree.show()

color
├── Green
│   ├── outline ___ Dashed
│   └── outline ___ Solid
├── Red
│   ├── has_dot ___ No
│   └── has_dot ___ Yes
└── Yellow

